# Solving the 1D heat equation

## The example

This example presents code to reproduce Example 5.1 from the Moin textbook

The PDE of interest is

$$
\frac{\partial T}{\partial t} = \alpha\frac{\partial^2 T}{\partial x^2} + (\pi^2-1)e^{-t}\sin{\pi x}\quad \text{for}\quad x\in[0, 1], \quad t\ge 0
$$

Initial condition: 
$$
T(x, 0) = \sin{\pi x}
$$

Boundary conditions:
$$
T(0, t) = T_{left} \quad\text{for}\quad t > 0
$$

$$
T(1, t) = T_{right} \quad\text{for}\quad t > 0
$$

Semi-discretization gives the following set of ODEs for inteiror nodes $j = 1, 2, ..., N_x-1$:

$$
\frac{dT_j}{dt}  = \beta(T_{j+1} - 2T_j + T_{j-1}) + (\pi^2-1)e^{-t}\sin{\pi x_j}
$$

where $\beta = \dfrac{\alpha}{\Delta x^2}$

The forward Euler udpate scheme for these nodes

$$
T_j^{n+1} = T_j^n + \Delta t\left[\beta(T_{j+1}^n - 2T_j^n + T_{j-1}^n) + (\pi^2-1)e^{-t_n}\sin{\pi x_j}\right]
$$


The code below implements the forward Euler method. In order to allow for adjusting the fixed boundary conditions, the code allows for specifying temperature conditions of <code>Tleft</code> and <code>Tright</code> at the left and right ends of the domain, respectively. Default values of each end condition are set to 0.

Try adjusting the time step, grid spacing, and boundary condition values. Does the solution behave as expected with these adjustments?

## Code Implementation

In [ ]:
# Required libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Space discretization
L = 1                       # Domain end point: 0 <= x <= L
dx = 0.05                   # Spatial grid spacing
Nx = round(L/dx)            # Nx + 1 grid points
x = np.linspace(0, L, Nx+1) # Grid points

# Time discretization
tend = 2                        # Ending time
dt = 0.001                      # Time step
Nt = int(tend/dt)               # Number of steps
t = np.linspace(0, dt*Nt, Nt+1) # List of solution times

# Diffusion constant and beta
alpha = 1
beta = alpha/dx**2

# Function for source term f(x)
f = lambda x, t: (np.pi**2 - 1)*np.exp(-t)*np.sin(np.pi*x)

# Print critical Forward Euler timestep info
print('dt =     ', dt)
print('dt_max = ', 0.5*dx**2/alpha)

# Initial condition
Tinit = np.sin(np.pi*x)

# Initialize solution variable
T = np.zeros((Nx+1, Nt+1))
T[:,0] = Tinit

# Boundary conditions (assumed fixed in time)
Tleft = 0
Tright = 0

# Forward Euler
for n in range(Nt):

    # Update interior nodes
    for i in range(1, Nx):
        T[i,n+1] = T[i,n] + dt*(beta*(T[i-1,n] - 2*T[i,n] + T[i+1,n]) + f(x[i], t[n]))

    # Update boundary nodes
    T[ 0, n+1] = Tleft
    T[Nx, n+1] = Tright

### Plotting

Below are commands to create
1. Plots of temperature distributions at several fixed times
2. Plots of temperature vs time for several fixed points on the grid
3. Animated plot of temperature distribution vs time


In [ ]:
"""Temperature distribution at several times"""
plt.figure()
tind = [0, round(0.25*Nt), round(0.5*Nt), round(0.75*Nt), Nt]
for n in tind:
    plt.plot(x, T[:,n], label="t = "+str(t[n]))
plt.title('Temperature vs position')
plt.xlabel('Position x')
plt.ylabel('T')
plt.legend()
plt.grid()

"""Solution at several domain points vs time"""
plt.figure()
xind = [0, round(0.25*Nx), round(0.5*Nx)]
for i in xind:
    plt.plot(t, T[i,:], label="x = "+str(x[i]))
plt.title('Temperature vs time')
plt.xlabel('Time t')
plt.ylabel('T')
plt.legend()
plt.grid()
plt.show()

"""Animated plot of temperature vs time"""
# Create the figure and axis
fig, ax = plt.subplots()
line, = ax.plot([], [], lw=2)

# Set the axis labels
ax.set_xlabel('Grid Points (x)')
ax.set_ylabel('Temperature (T)')
ax.set_title('Temperature Evolution Over Time')

# Set axis limits
ax.set_xlim(np.min(x), np.max(x))
ax.set_ylim(np.min(T), np.max(T))

# Initialization function to set up the background of each frame
def init():
    line.set_data([], [])
    return line,

# Animation function called sequentially to update the plot
def animate(j):
    line.set_data(x, T[:, j])
    ax.set_title(f'Time = {t[j]:.2f}')
    return line,

# Call the animator
N_every = 20
ani = animation.FuncAnimation(fig, animate, frames=range(0, len(t), N_every), init_func=init, blit=True, interval=100)
plt.close(fig)  # Close the static plot before displaying the animation

# To display the animation in the Jupyter notebook
HTML(ani.to_jshtml())

# Alternatively, if you want to save it as an MP4 or GIF, use:
# ani.save('temperature_animation.mp4', writer='ffmpeg')  # or 'temperature_animation.gif', writer='imagemagick'

## Code modification for flux boundary condition

To impose a flux of $q_0$ for time $t > 0$ we require

\begin{equation}
\frac{\partial T}{\partial x} = q_0 \quad\text{for}\quad t > 0
\end{equation}

Note that at $t = 0$, the flux condition is not imposed; instead, the initial condition takes precedence. <b>The flux condition "turns on" once we move past time 0.</b>

Applied at time $t_{n+1}$, a forward difference approximation to the spatial derivative gives
\begin{equation}
\frac{T_1^{n+1} - T_0^{n+1}}{\Delta x}= q_0
\end{equation}

Therefore the update rule for $T_0^{n+1}$ at every time step is
\begin{equation}
T_0^{n+1} = T_1^{n+1} - \Delta x q_0
\end{equation}

The code below is the same as the previous example except that it updates $T_0^{n+1}$ according to the formula above to impose the Neumann boundary condition during the time loop. The variable <code>q0</code> can be changed to change the boundary condition value from its default value of 0.
* <b>Important<b>: Note you must update the interior temperatures first in the time loop, since $T_0^{n+1}$ depends on $T_1^{n+1}$.

In [ ]:
# Space discretization
L = 1                       # Domain end point: 0 <= x <= L
dx = 0.05                   # Spatial grid spacing
Nx = round(L/dx)            # Nx + 1 grid points
x = np.linspace(0, L, Nx+1) # Grid points

# Time discretization
tend = 2                        # Ending time
dt = 0.001                      # Time step
Nt = int(tend/dt)               # Number of steps
t = np.linspace(0, dt*Nt, Nt+1) # List of solution times

# Diffusion constant and beta
alpha = 1
beta = alpha/dx**2

# Function for source term f(x)
f = lambda x, t: (np.pi**2 - 1)*np.exp(-t)*np.sin(np.pi*x)

# Print critical Forward Euler timestep info
print('dt =     ', dt)
print('dt_max = ', 0.5*dx**2/alpha)

# Initial condition
Tinit = np.sin(np.pi*x)

# Initialize solution variable
T = np.zeros((Nx+1, Nt+1))
T[:,0] = Tinit

# Boundary conditions (assumed fixed in time)
q0 =  0  #  Flux at left end
Tright  = 0  # Fixed temperature value at the right end

# Forward Euler
for n in range(Nt):

    # Update interior nodes
    for i in range(1, Nx):
        T[i,n+1] = T[i,n] + dt*(beta*(T[i-1,n] - 2*T[i,n] + T[i+1,n]) + f(x[i], t[n]))

    # Update boundary nodes
    T[ 0, n+1] = T[1, n+1] - dx*q0 # Flux BC
    T[Nx, n+1] = Tright # Fixed BC



### Plotting

Below are commands to create
1. Plots of temperature distributions at several fixed times
2. Plots of temperature vs time for several fixed points on the grid
3. Animated plot of temperature distribution vs time

In [ ]:
"""Temperature distribution at several times"""
plt.figure()
tind = [0, round(0.25*Nt), round(0.5*Nt), round(0.75*Nt), Nt]
for n in tind:
    plt.plot(x, T[:,n], label="t = "+str(t[n]))
plt.title('Temperature vs position')
plt.xlabel('Position x')
plt.ylabel('T')
plt.legend()
plt.grid()

"""Solution at several domain points vs time"""
plt.figure()
xind = [0, round(0.25*Nx), round(0.5*Nx)]
for i in xind:
    plt.plot(t, T[i,:], label="x = "+str(x[i]))
plt.title('Temperature vs time')
plt.xlabel('Time t')
plt.ylabel('T')
plt.legend()
plt.grid()
plt.show()

"""Animated plot of temperature vs time"""
# Create the figure and axis
fig, ax = plt.subplots()
line, = ax.plot([], [], lw=2)

# Set the axis labels
ax.set_xlabel('Grid Points (x)')
ax.set_ylabel('Temperature (T)')
ax.set_title('Temperature Evolution Over Time')

# Set axis limits
ax.set_xlim(np.min(x), np.max(x))
ax.set_ylim(np.min(T), np.max(T))

# Initialization function to set up the background of each frame
def init():
    line.set_data([], [])
    return line,

# Animation function called sequentially to update the plot
def animate(j):
    line.set_data(x, T[:, j])
    ax.set_title(f'Time = {t[j]:.2f}')
    return line,

# Call the animator
N_every = 10
ani = animation.FuncAnimation(fig, animate, frames=range(0, len(t), N_every), init_func=init, blit=True, interval=100)
plt.close(fig)  # Close the static plot before displaying the animation

# To display the animation in the Jupyter notebook
HTML(ani.to_jshtml())

# Alternatively, if you want to save it as an MP4 or GIF, use:
# ani.save('temperature_animation.mp4', writer='ffmpeg')  # or 'temperature_animation.gif', writer='imagemagick'